In [1]:
%config InlineBackend.figure_format = 'retina' #to make plots look sharper

In [2]:
import pickle
from glob import glob
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.graph_objs as go
from plotly.offline import plot
import matplotlib.cm as cm

#import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV,Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import MultiTaskElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from itertools import cycle

In [3]:
def fcsvtodf(fcsv_path):
  """takes single subject path to .fscv file and converts to a dataframe"""
  # extract the subject ID from the file path
  subject_id = re.search(r'(sub-\w+)', fcsv_path).group(1)
  #read in fcsv file with coordiantes and some other fields
  df_raw = pd.read_table(fcsv_path,sep=',',header=2)
  #use number of row in fcsv to make number points
  colnames = [f'{axis}_{i % int(df_raw.shape[0]) + 1}' for axis in ['x', 'y', 'z'] for i in range(int(df_raw.shape[0]))]
  #extract the x, y, z coordiantes and store them in data science friendly format (i.e., features in cols and subject in rows)
  df_xyz = df_raw[['x','y','z']].melt().transpose()
  #reassign features to be descriptive of coordinate
  df_xyz.columns = colnames
  #clean dataframe and pin correct subject name
  df_xyz_clean = df_xyz.drop('variable', axis= 0)
  df_xyz_clean = df_xyz_clean.rename(index={'value': subject_id})
  df_xyz_clean = df_xyz_clean.astype(float)
  return df_xyz_clean , df_raw.shape[0]

def afid_distance (path,x,y):
  "takes two afids as input (in short hand label i.e., AC) and computes the euclidean distance between the two afids, need to also provide a dataframe which has the x,y,z of afids (as columns) for all subjects/templates"
  fid_dic = {1: 'AC', 2: 'PC',3: 'ICS',4: 'PMJ',5: 'SIPF',6: 'RSLMS',7: 'LSLMS',8: 'RILMS',9: 'LILMS',10: 'CUL',11: 'IMS',12: 'RMB',13: 'LMB',14: 'PG',15: 'RLVAC',16: 'LLVAC',17: 'RLVPC',18: 'LLVPC',19: 'GENU',20: 'SPLE',21: 'RALTH',22: 'LALTH',23: 'RSAMTH',24: 'LSAMTH',25: 'RIAMTH',26: 'LIAMTH',27: 'RIGO',28: 'LIGO',29: 'RVOH',30: 'LVOH',31: 'ROSF',32: 'LOSF'}

  #check to see if user references afids as string (i.e., AC as 'AC')
  if type(x) == str:
      # print('user assigned afids as a str')
      for i,e in fid_dic.items():
          if e == x:
            afid1 = i
          if e == y:
            afid2 = i

    #if user references afids as an index (i.e., AC as 1)
  else:
    # print('user assigned afids as an index')
    afid1 = x
    afid2 = y

  df = fcsvtodf(path)[0]

  #perform the subtraction b.w two afids in a dataframe
  x = df[f"x_{afid1}"] - df[f"x_{afid2}"]
  y = df[f"y_{afid1}"] - df[f"y_{afid2}"]
  z = df[f"z_{afid1}"] - df[f"z_{afid2}"]
  xyz_diff = np.array([x[0],y[0],z[0]])
  distance = np.sqrt(xyz_diff[0]**2 + xyz_diff[1]**2 + xyz_diff[2]**2)
  return xyz_diff, distance

def acpcmatrix(path, midline):
  '''takes as input one midline point (any of the 10 midline points from the afids protocol)'''
  # define the antero and posterior axis
  acpc = afid_distance(path, 'AC', 'PC')[0]  # subtracting ac and pc yields a line along AC and PC
  yAxis = acpc / np.linalg.norm(acpc)  # gives the length of AC and PC

  # Lateral axis
  lataxis = afid_distance(path, midline, 'AC')[0]
  xAxis = np.cross(yAxis, lataxis)
  xAxis /= np.linalg.norm(xAxis)

  # Rostrocaudal axis
  zAxis = np.cross(xAxis, yAxis)
  zAxis /= np.linalg.norm(zAxis)

  # Rotation matrix
  rotation = np.vstack([xAxis, yAxis, zAxis])
  point = np.array([0, 0, 0])

  # Build 4x4 matrix
  matrix = np.eye(4)
  matrix[:3, :3] = rotation
  matrix[:3, 3] = point

  if matrix[0][0] < 0:
    matrix = matrix * np.array([[-1,-1,-1,0],[1,1,1,0],[-1,-1,-1,0],[0,0,0,1]]) #correction for matrix is midpoint placed below acpc (i.e., at PMJ)

  return matrix

def mcp_origin(df_afids):
  """sets MCP as the origin point for each of the subjects"""
  # extract MCP coordinates; defined as average point between AC and PC
  MCPx = (df_afids['x_1'] + df_afids['x_2'])/2
  MCPy = (df_afids['y_1'] + df_afids['y_2'])/2
  MCPz = (df_afids['z_1'] + df_afids['z_2'])/2

  # subtract MCP coordinates from afids at appropriate coords
  df_afids_mcpx = df_afids.transpose()[0:32] - MCPx
  df_afids_mcpy= df_afids.transpose()[32:64] - MCPy
  df_afids_mcpz= df_afids.transpose()[64:98] - MCPz

  #concat the three coords and take transpose
  frames = [df_afids_mcpx, df_afids_mcpy, df_afids_mcpz]
  df_afids_mcp = pd.concat(frames)
  df_afids_ori_mcp = df_afids_mcp.transpose()
  df_afids_ori_mcp = df_afids_ori_mcp.astype(float)
  return df_afids_ori_mcp , (np.array([MCPx.to_numpy(),MCPy.to_numpy(),MCPz.to_numpy()]))

def seg_to_fcsv(weighted_centroids, fcsv_template, fcsv_output):
    # Read in fcsv template
    with open(fcsv_template, "r") as f:
        fcsv = [line.strip() for line in f]

    # Loop over fiducials
    for fid in range(1, weighted_centroids.shape[0]+1):
        # Update fcsv, skipping header
        line_idx = fid + 2
        centroid_idx = fid - 1
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_x", str(weighted_centroids[centroid_idx][0])
        )
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_y", str(weighted_centroids[centroid_idx][1])
        )
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_z", str(weighted_centroids[centroid_idx][2])
        )

    # Write output fcsv
    with open(fcsv_output, "w") as f:
        f.write("\n".join(line for line in fcsv))

def make_zero(num, threshold=0.0001):
    if abs(num) < threshold:
        return 0
    else:
        return num

def remove_outliers(df, cols, threshold=3.5):
    """
    Remove rows from a DataFrame based on a modified Z-score for a given column(s).
    """
    for col in cols:
        median = np.median(df[col])
        mad = np.median(np.abs(df[col] - median))
        if mad == 0:
            mad = 1.0
        z_scores = 0.6745 * (df[col] - median) / mad
        df = df.loc[abs(z_scores) < threshold]
    return df

def create_list(n):
    numbers = list(range(1, n + 1))  # Create a list of numbers from 1 to n
    repeated_numbers = cycle(numbers)  # Create a cycle iterator for the numbers
    result = [next(repeated_numbers) for _ in range(n * 2)]  # Generate n*2 elements by cycling through the numbers
    return result

def extract_subject_id_from_path(afid_file):
    """Extracts the subject ID from the afid_file path."""
    match = re.search(r'sub-\w+', afid_file)
    return match.group(0) if match else None

def get_dataset_name(file_path, level=8):
    """Extract the dataset name from the file path based on the specified level."""
    parts = file_path.split(os.sep)
    if len(parts) > level:
        return parts[level]
    return "Unknown"

def plot_multiple_files(file_list, out_file, rotation=False, trace_lines=False):
    traces = []
    all_afids = []
    all_points = pd.DataFrame()

    # Gather all AFIDs from all files for consistent coloring
    for file_path in file_list:
        data = pd.read_csv(file_path, skiprows=2)
        all_afids.extend(data['desc'].unique())
        data['file'] = file_path  # Add a column to track the source file
        data['dataset'] = get_dataset_name(file_path)  # Extract dataset name based on path level
        all_points = pd.concat([all_points, data], ignore_index=True)

    # Get unique AFIDs and assign colors
    unique_afids = list(set(all_afids))
    afid_labels = {afid: str(i + 1) for i, afid in enumerate(unique_afids)}  # Numbering AFIDs
    color_map = cm.get_cmap('tab20', len(unique_afids))  # Use tab20 colormap for wider range
    afid_colors = {afid: color_map(i) for i, afid in enumerate(unique_afids)}

    # Convert colors to RGBA format
    afid_colors = {afid: f'rgba({int(r*255)}, {int(g*255)}, {int(b*255)}, {a})' 
                   for afid, (r, g, b, a) in afid_colors.items()}

    # Calculate centroid for each AFID and keep the descriptions
    centroids = all_points.groupby('desc').agg({'x': 'mean', 'y': 'mean', 'z': 'mean'}).reset_index()

    # Get the unique dataset names
    datasets = all_points['dataset'].unique()

    # Create a dummy trace for each dataset to allow toggling all subjects in the dataset
    for dataset in datasets:
        # Create a dummy scatter trace to act as the dataset header
        dataset_trace = go.Scatter3d(
            x=[None], y=[None], z=[None],  # Invisible placeholder
            mode='markers',
            marker=dict(size=1),
            showlegend=True,
            name=f'{dataset}',  # Name the trace as the dataset
            legendgroup=dataset,  # Group by dataset
            hoverinfo='none'
        )
        traces.append(dataset_trace)

    # Create traces for original points, grouped by dataset
    for file_path in file_list:
        data = all_points[all_points['file'] == file_path]
        x_coords = data['x']
        y_coords = data['y']
        z_coords = data['z']
        afids = data['desc']
        dataset_name = data['dataset'].iloc[0]  # Get dataset name for grouping

        # Get colors for each point based on its AFID
        colors = [afid_colors[afid] for afid in afids]

        # Create a scatter trace for each subject, grouped by dataset
        trace = go.Scatter3d(
            x=x_coords,
            y=y_coords,
            z=z_coords,
            mode='markers',
            marker=dict(
                size=7,
                color=colors,
                opacity=0.99,
                line=dict(width=2)
            ),
            name=f'{file_path.split("/")[-1]}',
            legendgroup=dataset_name,  # Group by dataset name
            showlegend=True
        )
        traces.append(trace)

    # Add centroid markers with the AFID descriptions as labels
    centroid_trace = go.Scatter3d(
        x=centroids['x'],
        y=centroids['y'],
        z=centroids['z'],
        mode='markers',
        text=centroids['desc'],  # Label with the AFID descriptions
        textposition='top center',
        marker=dict(
            size=7, 
            color='black',
            opacity=0.99,
            symbol='diamond',
            line=dict(width=2)
        ),
        name='Centroid',
        showlegend=True
    )
    traces.append(centroid_trace)

    if trace_lines:
        # Optionally, add lines connecting the centroids
        line_trace = go.Scatter3d(
            x=np.repeat(centroids['x'], 2)[1:-1],
            y=np.repeat(centroids['y'], 2)[1:-1],
            z=np.repeat(centroids['z'], 2)[1:-1],
            mode='lines',
            line=dict(color='grey', width=2),
            name='Connections',
            showlegend=True
        )
        traces.append(line_trace)

    # Set layout with more visual enhancements
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X Coordinate', backgroundcolor='rgb(240, 240, 240)', showgrid=True, gridcolor='white'),
            yaxis=dict(title='Y Coordinate', backgroundcolor='rgb(240, 240, 240)', showgrid=True, gridcolor='white'),
            zaxis=dict(title='Z Coordinate', backgroundcolor='rgb(240, 240, 240)', showgrid=True, gridcolor='white'),
            aspectmode='data'  # Keep the aspect ratio based on the data
        ),
        title="3D Interactive Plot of Fiducial Points with Centroids",
        margin=dict(l=0, r=0, b=0, t=30),
        legend=dict(title="Legend", itemsizing='constant', groupclick="toggleitem"),  # Allows toggling entire datasets
        scene_camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        ),
        hovermode='closest'
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)

    if rotation:
        # Prepare animation frames for a smooth 360-degree rotation
        frames = [go.Frame(layout=dict(scene_camera=dict(eye=dict(x=1.5*np.cos(np.radians(angle)), 
                                                                   y=1.5*np.sin(np.radians(angle)), 
                                                                   z=2))))
                  for angle in range(0, 360, 60)]  # Frames for smoother animation

        # Add the play button for the rotation animation
        fig.update_layout(
            updatemenus=[dict(type='buttons', showactive=False,
                              buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)])])],
        )

        # Attach frames to the figure
        fig.frames = frames

    # Display the interactive plot
    plot(fig, filename=out_file)

    return out_file

In [10]:
#load coordiante data from the afids data dataset for prelim analysis
afid_fcsv = sorted(glob(f'/Users/alaataha/Desktop/GitHub/afids-pred/coord_data/AFIDs/*/afids_groundtruth/sub-*/anat/sub-*desc-groundtruth_afids.fcsv'))
print(f'number of fcsv: {len(afid_fcsv)}')
stn_df = pd.read_csv('/Users/alaataha/Desktop/GitHub/afids-pred/coord_data/STN/STN_all.csv')
print(f'number of segmentations: {len(stn_df)//2}')

number of fcsv: 200
number of segmentations: 70


In [11]:
midpoint='PMJ'
# Need to run this once to tranform coordinates into ACPC space and save outputs.
for afid_file in afid_fcsv:
  xfm_txt = acpcmatrix(afid_file,midpoint).T #this is the 4x4 transform, we choose a third point (i.e., "midpoint") to define the midline plane
  subject_id = extract_subject_id_from_path(afid_file)

  stn_df_subjects = stn_df[['subjid']].to_numpy()
  matching_entries = [subjid[0] for subjid in stn_df_subjects if subject_id in subjid[0]]
  
  coords_stn = stn_df[stn_df['subjid'].str.contains(subject_id, na=False)][["centroid_x","centroid_y","centroid_z"]].to_numpy()
  print (coords_stn)
  # # applying for AFIDs since there are 32 in one file need to loop through later
  # fcsv_df = pd.read_table(afid_file, sep=",", header=2)
  # coords = fcsv_df[["x", "y", "z"]].to_numpy()
  # tcoords = np.zeros(coords.shape)
  # tcoords_stn = np.zeros(coords_stn.shape)


  # for i in range(len(coords)):
  #   vec = np.hstack([coords[i, :], 1]) 
  #   tvec = np.linalg.inv(xfm_txt) @ vec.T #applying tranformations to coordiantes is done in the opposite direction
  #   tcoords[i, :] = tvec[:3]


  # for n in range(len(coords_stn.shape)):
  #   vec_stn = np.hstack(coords_stn[n,:],1)
  #   tvec_stn = np.linalg.inv(xfm_txt) @ vec_stn.T #applying tranformations to coordiantes is done in the opposite direction
  #   tcoords_stn[i, :] = tvec_stn[:3]



  # mcp = (tcoords[0] + tcoords[1])/2
  # tcoords = tcoords - mcp
  # tcoords_stn = tcoords_stn - mcp

  # seg_to_fcsv(tcoords_stn, '/Users/alaataha/Desktop/GitHub/afids-pred/resources/stn_template.fcsv', f'{os.path.split(afid_file)[0]}/{os.path.splitext(os.path.split(afid_file)[1])[0]}mcp_{midpoint}_stn.fcsv') #save outputs for comaprision ##saves output AFIDs in ACPC transformed space
  # seg_to_fcsv(tcoords, '/Users/alaataha/Desktop/GitHub/afids-pred/resources/afids_template.fcsv', f'{os.path.split(afid_file)[0]}/{os.path.splitext(os.path.split(afid_file)[1])[0]}mcp_{midpoint}.fcsv') #save outputs for comaprision ##saves output AFIDs in ACPC transformed space

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipyker

[[-10.2717   25.2001   13.5802 ]
 [  9.95442  25.1775   12.0361 ]]
[[-8.51599 19.1185   8.33398]
 [11.4459  20.2445   8.20792]]
[[ -6.75441  20.9804  -17.6547 ]
 [ 12.9332   19.7491  -16.6896 ]]
[[-11.6816   37.1169    4.72863]
 [ 11.0437   41.1537    6.34567]]
[[-7.38121 30.1683  -7.23315]
 [10.0886  31.9149  -7.23698]]
[[-9.93778  20.2451    0.821535]
 [10.4217   19.0043    1.57182 ]]
[[ -9.33155  34.3658  -10.2913 ]
 [ 13.2538   34.4213  -11.3005 ]]
[[-8.80798 31.7085  -9.14269]
 [11.2342  30.2615  -9.14065]]
[[-7.2077  28.8021   7.8062 ]
 [13.4719  27.151    7.39882]]
[[-8.91285 32.6917  -4.7569 ]
 [13.9707  35.068   -6.06229]]
[[-10.2717   25.2001   13.5802 ]
 [  9.95442  25.1775   12.0361 ]]
[[-8.51599 19.1185   8.33398]
 [11.4459  20.2445   8.20792]]
[[ -6.75441  20.9804  -17.6547 ]
 [ 12.9332   19.7491  -16.6896 ]]
[[-11.6816   37.1169    4.72863]
 [ 11.0437   41.1537    6.34567]]
[[-7.38121 30.1683  -7.23315]
 [10.0886  31.9149  -7.23698]]
[[-9.93778  20.2451    0.821535]
 [10

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipyker

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipyker

...

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipyker

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[-10.58516    3.77617  -24.466611]
 [ 11.125667   4.356695 -24.843787]]
[[ -7.404793  -8.916362 -42.775581]
 [ 11.319304  -8.859449 -43.244099]]
[[ -4.495196 -11.190258 -21.19475 ]
 [ 15.227568  -9.519651 -19.625608]]
[[-10.477353  -3.961219 -35.862049]
 [ 11.708408  -3.341259 -35.256687]]
[[ -7.323238  -6.269578 -19.840917]
 [ 14.563116  -7.46987  -20.06164 ]]
[[ -7.713447  -0.105135 -17.774627]
 [ 10.466467  -0.689109 -18.546331]]
[[ -0.127863  -4.03458  -49.483202]
 [ 19.644517  -4.289928 -50.602979]]
[[ -9.162996  -8.430076 -15.969515]
 [ 12.555344  -7.671839 -15.430596]]
[[-10.424057  -0.546425 -21.549252]
 [ 11.03604   -0.619595 -20.567055]]
[[ -8.075046  -9.219937 -46.372968]
 [ 12.090619  -9.248163 -46.536617]]
[[ -8.136833 -11.491609 -28.758228]
 [ 11.586049 -10.890252 -29.712807]]
[[ -7.022659 -11.502645 -30.142731]
 [ 12.327568 -10.659645 -30.098708]]
[[ -9.488039  -4.180766 -20.405565]
 [ 10.093708  -3.11272

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipyker

[[ -7.035155  -6.571819 -27.540147]
 [ 12.860213  -7.041335 -28.103303]]
[[ -5.801676  -8.355791 -17.292034]
 [ 14.774889  -7.078695 -17.155985]]
[[ -5.449196 -17.918602 -26.003297]
 [ 14.824224 -18.67127  -26.901438]]
[[ -9.209735  -8.74766  -15.314637]
 [ 12.754985  -9.854491 -15.189108]]
[[ -9.141575 -11.2797   -19.323039]
 [  9.595732 -10.658413 -19.351108]]
[[ -9.321443  -9.67318  -38.931102]
 [ 13.732391  -8.177588 -39.051847]]
[[ -6.981971  -4.814656 -45.091325]
 [ 10.642993  -6.0206   -45.175778]]
[[ -4.747338  -1.846144 -22.498194]
 [ 15.837826  -1.540674 -24.51905 ]]
[[-10.281261 -11.544735 -36.713667]
 [ 11.89957  -11.008373 -37.078262]]
[[ -6.31632  -13.071823 -32.962243]
 [ 14.303336 -13.023128 -33.375165]]
[[ -8.937061 -10.56153  -12.59267 ]
 [ 10.869138 -10.562614 -13.004917]]
[[ -9.282507 -16.221557 -14.028193]
 [ 12.057277 -12.635877 -13.778173]]
[[ -7.37945  -9.57445 -21.2077 ]
 [ 11.4736   -9.51633 -20.5994 ]]
[[ -7.19424 -16.5449  -35.3598 ]
 [ 11.2981  -17.0396  -3

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_4429/106248827.py:44: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipyker

In [22]:
file_paths = afid_fcsv
# Plot the files together
plot_multiple_files(file_paths, out_file='/Users/alaataha/Desktop/GitHub/afids-pred/interactive_AFIDs_plot.html', rotation=True)

/var/folders/1_/pqzl0fjs2595xmp99j42cvmc0000gn/T/ipykernel_34350/1956625964.py:24: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



'/Users/alaataha/Desktop/GitHub/afids-pred/interactive_AFIDs_plot.html'